# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [5]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)  # me muestre todas las columnas

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [6]:
orders=pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [140]:
ord_byclient=orders.groupby("CustomerID").sum("amount_spent")
ord_byclient=ord_byclient.reset_index()

stats_byclient=ord_byclient.describe().T

Q95=ord_byclient["amount_spent"].quantile(0.95)

top95=ord_byclient[ord_byclient["amount_spent"] > Q95]

In [141]:
ord_byclient1 = ord_byclient.copy()

ord_byclient1["class"]="Preferred"
ord_byclient1["class"][ord_byclient1["amount_spent"] > Q95]="VIP"
ord_byclient1["class"].value_counts()

Preferred    4122
VIP           217
Name: class, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [165]:
orders_uniclient=orders[["CustomerID", "Country"]]
orders_uniclient=orders_uniclient.drop_duplicates()

ord_byclient1=ord_byclient1[["CustomerID", "class"]]

thedf=pd.merge(ord_byclient1, orders_uniclient, on="CustomerID")

VIP_countries = thedf.pivot_table(index='Country', columns='class', values='class', aggfunc='size', fill_value=0)

display(VIP_countries.sort_values("VIP", ascending=False)[:5])

TOP=(list(VIP_countries.sort_values("VIP", ascending=False).index)[0])

print(f'{TOP} has the most VIP Customers')


#thedf.groupby("Country")["class"].value_counts("class")


class,Preferred,VIP
Country,,
United Kingdom,3744,177
Germany,84,10
France,78,9
Switzerland,18,3
Australia,7,2


United Kingdom has the most VIP Customers


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [158]:
display(VIP_countries.sort_values("Preferred", ascending=False)[:5])

TOP=(list(VIP_countries.sort_values("Preferred", ascending=False).index)[0])

print(f'{TOP} has the most VIP+Preferred Customers combined')

class,Preferred,VIP
Country,,
United Kingdom,3744,177
Germany,84,10
France,78,9
Spain,28,2
Belgium,24,1


United Kingdom has the most VIP+Preferred Customers combined
